<a href="https://colab.research.google.com/github/alexsalman/DeepLearningAssignments/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Assignment 1

**DUE: Sunday October 10 at 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/).

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Runtime→→Restart runtime) and then run all cells (in the menubar, select Runtime→→Run All).

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
NAME = ""
STUDENT_ID = ""

## Covid-19 Case Prediction

In this Assignment you will learn:


*   Solving a Regression problem using Deep Neural Networks
*   Basic Deep Learning training tips such as hyperparameter tuning and optimization
*   Get familiar with TensorFlow

**Task:** Given the survey results in the past 2 days, in a specific State, predict the percentage of newly tested positive cases for the 3rd day (in percentage).

Data includes the following information (each row includes features for all three days): 

*   States - Through 40 one-hot encoded State information
*   Covid-like illnesses (cli, etc.) (4 features - in percentage)
*   Behavior indicators (wearing masks, etc.) (8 features - in percentage)
*   Mental health indicators (depressed, etc.) (5 features - in percentage)

**Evaluation Metric**: Root Mean Squared Error





In [ ]:
import numpy as np
import pandas as pd
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1wrk4vvTxnp1_3Ftx7qp9uo4IUNBWPAA8"}) 
downloaded.GetContentFile('covid.csv')

# Finally we can actually read in the data.
data = pd.read_csv('covid.csv')

### Part 1) Data preparation

In this part you will complete the following steps to prepare your data for training:

*   Understand the data,
*   Separate label from features,
*   Normalize features, and
*   Split the data into train and test (and later validation)


In [ ]:
## YOUR CODE HERE ##

### Part 2) Hyperparameeters and neural network training

In this part you will be implementing a neural network and will investigate the impact of hyperparameters in model's performance.

1.   Train a neural network with 2 layers (one hidden layer and one output layer).
2.   Search for the most appropriate hyperparameter setting for your model. These hyperparameters include but not limited to (number of epochs, batch size, learning rate, optimizer, momentum).
3. Describe your findings in the text cell below.

In [ ]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 3) Understanding neural network architecture

In this part you will experiment with different deep neural network architectures to understand the impact of the architecture on model's accuracy and generalization.

1.   Run an experiment on a 2-layer neural network to investigate the impact of the number of neurons in the hidden layer on model's performance on the training and test data. Plot the result of your experiment.

2. Run the same experiment as the previous part but use depth 3 (2 hidden layers and one output layer). Plot the result of your experiment.

3.   Run an experiment on the impact of the depth of the model on model's performance on the training and test data. For this experiment, use 5 neurons in each hidden layer. Plot the result of your experiment.

4. Describe your findings in the text cell below.




In [ ]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 4) Feature selection and final results

Since the number of feeatures will impact model's complexity and that not all features contribute equally to the model's performance, in this part you are asked to perform feature selection, model training, and reporting final results.

Your tasks are:
1.   Based on the experiments you have conducted (and looking into the weights associated to features), perform feature selection.
2. Train an optimum neural network architecture (you will need to do some experiments to find it) with appropriate hyperparameters (you will need to do hyperparameter tuning).
3. Report the Loss of your model on both training and test data. In addition, plot the prediction vs. target labels for both training and test data.


In [ ]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 5) State-specific prediction

Thinking about the structure of the data, please provide a description of technique(s) you can use to perform predictions for a given State based on its own data (which reduces the bias in prediction). 

[YOUR ANSWER HERE]

### (BONUS) Part 6) Implement one of your responses to the previous part for an extra credit

To receive an extra credit, you should show the performance boost you observed for the State-specific architecture. 

In [ ]:
## YOUR CODE HERE ##